In [ ]:
import ee
import pandas as pd
print("Earth Engine API imported successfully!")

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-damyadharmawardhana')

In [ ]:
roi = ee.Geometry.Polygon([
    [[7.62606, 79.80086],
    #  [7.62470, 79.86712],
    #  [7.59595, 79.86712],
    #  [7.59731, 79.78592]
     ]
])
start_date = '1990-01-01'
end_date = '2024-12-31'

# Filter dataset
dataset = ee.ImageCollection('ECMWF/ERA5/MONTHLY') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)

# Calculate monthly mean temperature
def calculate_mean(image):
    date = ee.Date(image.get('system:time_start'))
    month = date.format('YYYY-MM')
    mean_temp = image.select('mean_2m_air_temperature') \
        .reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi,
            scale=1000,
            maxPixels=1e13
        ).get('mean_2m_air_temperature')
    return ee.Feature(None, {'date': month, 'mean_temp': mean_temp})

# Apply mapping
monthly_mean_features = dataset.map(calculate_mean)

# Convert the FeatureCollection to a DataFrame
feature_list = monthly_mean_features.getInfo()['features']
data = [{'date': feature['properties']['date'], 'mean_temp': feature['properties']['mean_temp']} for feature in feature_list]

import pandas as pd
df = pd.DataFrame(data)

In [ ]:
print(df)

In [ ]:
#Debugging Missing Values
print(dataset.first().getInfo())  # Inspect the first image for variable names

#### Verify Dataset Coverage

In [ ]:
print(dataset.size().getInfo()) 

#### Revisit Mapping and System Properties

In [ ]:
def check_properties(image):
    return ee.Feature(None, {
        'id': image.id(),
        'system_time_start': image.get('system:time_start')
    })

properties = dataset.map(check_properties).getInfo()

# Print to inspect missing properties
for feature in properties['features']:
    print(feature['properties'])

#### Test with a Simpler ROI

In [ ]:
roi_test = ee.Geometry.Rectangle([79.6, 7.5, 79.9, 8.0])  # Larger rectangle
dataset_test = ee.ImageCollection('ECMWF/ERA5/MONTHLY') \
    .filterBounds(roi_test) \
    .filterDate(start_date, end_date)

print(dataset_test.size().getInfo())  # Should return a number > 0

In [ ]:
#Expand ROI
expanded_roi = roi.buffer(10000)  # Buffer by 10 km
dataset_expanded = ee.ImageCollection('ECMWF/ERA5/MONTHLY') \
    .filterBounds(expanded_roi) \
    .filterDate(start_date, end_date)

print(dataset_expanded.size().getInfo())

#### Debugging the Time Range

In [ ]:
dataset_debug = ee.ImageCollection('ECMWF/ERA5/MONTHLY') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-31')  # One year for testing

print(dataset_debug.size().getInfo())  # Check if data exists for this time range

#### Yearly data

In [ ]:
import ee
import pandas as pd
print("Earth Engine API imported successfully!")

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-damyadharmawardhana')

In [24]:
mender_dispalcement = pd.read_csv('meander_displacement.csv')

In [ ]:
# Define the area of interest (ROI)
aoi = ee.Geometry.Polygon([
    [[79.80086, 7.62606], [79.86712, 7.62470],
     [79.86712, 7.59595], [79.78592, 7.59731], [79.80086, 7.62606]]
])

# Define the time range
start_date = '1980-01-01'
end_date = '2024-10-31'

# Load the ERA5 Daily dataset
era5 = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .filterDate(start_date, end_date) \
    .select('mean_2m_air_temperature')

# Function to reduce the image collection to mean temperature and date
def extract_temperature(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    temperature = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000,
        maxPixels=1e9
    ).get('mean_2m_air_temperature')
    return ee.Feature(None, {'date': date, 'temperature': temperature})

# Apply the function to the image collection
temperature_series = era5.map(extract_temperature)

# Export the data to a CSV file in Google Drive
task = ee.batch.Export.table.toDrive(
    collection=temperature_series,
    description='Temperature_Data_Export',
    fileFormat='CSV',
    folder='GEE_Temperature'  # You can choose your Google Drive folder here
)
task.start()

print("Export started. Check your Google Drive after processing.")

In [ ]:
# Define the area of interest (ROI)
aoi = ee.Geometry.Polygon([
    [[79.80086, 7.62606], [79.86712, 7.62470],
     [79.86712, 7.59595], [79.78592, 7.59731], [79.80086, 7.62606]]
])

# Define the time range
start_date = '1980-01-01'
end_date = '2024-10-31'

# Load the ERA5 Daily dataset
era5 = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .filterDate(start_date, end_date)

# Function to reduce the image collection to mean rainfall and date
def extract_rainfall(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    rainfall = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000,
        maxPixels=1e9
    ).get('total_precipitation')  # Adjust to the correct rainfall band
    return ee.Feature(None, {'date': date, 'rainfall': rainfall})

# Select the rainfall band and apply the function
rainfall_series = era5.select('total_precipitation').map(extract_rainfall)

# Export the data to a CSV file in Google Drive
task = ee.batch.Export.table.toDrive(
    collection=rainfall_series,
    description='Rainfall_Data_Export',
    fileFormat='CSV',
    folder='GEE_Rainfall'  # You can choose your Google Drive folder here
)
task.start()

print("Rainfall export started. Check your Google Drive after processing.")

In [5]:
Temp_data = pd.read_csv('Temperature_Data_Export.csv')

In [6]:
rainfall_data = pd.read_csv('Rainfall_Data_Export.csv')

In [12]:
temperature_data = pd.read_csv('Temperature_Data_Export.csv')  # File with date and temperature
rainfall_data = pd.read_csv('Rainfall_Data_Export.csv')  # File with date and rainfall

# Ensure both datasets have 'date' in the same format
temperature_data['date'] = pd.to_datetime(temperature_data['date'])
rainfall_data['date'] = pd.to_datetime(rainfall_data['date'])

# Merge the two datasets on the 'date' column
temp_rainfall_data = pd.merge(temperature_data, rainfall_data, on='date', how='inner')

In [ ]:
temp_rainfall_data

In [18]:
temp_rainfall_data = temp_rainfall_data.drop(['.geo_x', '.geo_y', 'system:index_y', 'system:index_x'], axis=1)

In [ ]:
temp_rainfall_data

In [ ]:
mender_dispalcement

In [26]:
# Drop the first row (contains headers as data)
mender_dispalcement = mender_dispalcement.iloc[1:].reset_index(drop=True)

# Rename columns for easier access
mender_dispalcement.columns = ['image', 'c1_x', 'c1_y', 'c1_dist', 'c2_x', 'c2_y', 'c2_dist', 
                'c3_x', 'c3_y', 'c3_dist', 'c4_x', 'c4_y', 'c4_dist','c5_x', 'c5_y', 'c5_dist','c6_x', 'c6_y', 'c6_dist','c7_x', 'c7_y', 'c7_dist','c8_x', 'c8_y','c8_dist'] 

# Convert numeric columns to proper data types
numeric_cols = mender_dispalcement.columns[1:]  # Skip the 'image' column
mender_dispalcement[numeric_cols] = mender_dispalcement[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Handle missing data if necessary
mender_dispalcement = mender_dispalcement.fillna(0)  # Example: Fill NaN with 0

In [27]:
# Extract the date from the 'image' column and convert to datetime format
mender_dispalcement['date'] = mender_dispalcement['image'].str.extract(r'(\d{4}_\d{2}_\d{2})')[0]  # Extract date part
mender_dispalcement['date'] = pd.to_datetime(mender_dispalcement['date'], format='%Y_%m_%d')  # Convert to datetime

# Drop the original 'image' column if not needed
mender_dispalcement = mender_dispalcement.drop(columns=['image'])

In [ ]:
mender_dispalcement